In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from googletrans import Translator

In [28]:
df = pd.read_csv('text-data.csv', encoding="ISO-8859-1")
df.head()

,sentiment,text,translated
0,neutral,"According to Gran , the company has no plans t...",False
1,neutral,Technopolis plans to develop in stages an area...,False
2,negative,The international electronic industry company ...,False
3,positive,With the new production plant the company woul...,False
4,positive,According to the company 's updated strategy f...,False


In [29]:
translator = Translator()

In [30]:
def translate_batch(texts):
    translations = translator.translate(texts, dest='id')
    return [translation.text for translation in translations]

In [31]:
batch_size = 100  # Adjust batch size as needed
num_batches = len(df) // batch_size + (1 if len(df) % batch_size else 0)

In [32]:
start_index = df.loc[~df['translated']].index.min() if not df['translated'].all() else len(df)

# Calculate the starting batch based on the start_index
start_batch = start_index // batch_size

for i in range(start_batch, num_batches):
    # Calculate batch indices considering the DataFrame might not be perfectly divisible by batch_size
    batch_indices = range(i*batch_size, min((i+1)*batch_size, len(df)))
    # Select only untranslated texts in the current batch
    untranslated_batch = df.loc[batch_indices].loc[~df['translated'], 'text'].tolist()
    
    if untranslated_batch:  # Check if there are any untranslated texts left
        translated_batch = translate_batch(untranslated_batch)
        # Update the DataFrame with translated texts and mark them as translated
        # This assumes the order of texts returned by translate_batch matches the order of texts sent
        untranslated_indices = df.loc[batch_indices][~df.loc[batch_indices]['translated']].index
        for j, idx in enumerate(untranslated_indices):
            df.at[idx, 'text'] = translated_batch[j]
            df.at[idx, 'translated'] = True
        # Optionally, save the DataFrame to a file or database here to persist changes

In [33]:
translations = translator.translate(['The quick brown fox', 'jumps over', 'the lazy dog'], dest='id')
for translation in translations:
    print(translation.origin, ' -> ', translation.text)

The quick brown fox  ->  Rubah coklat yang cepat
jumps over  ->  melompati
the lazy dog  ->  anjing pemalas


In [34]:
df.to_csv('translated-text-data.csv', index=False)